<a href="https://colab.research.google.com/github/uarmirimau/PIAIC-ISB-Batch58-Projects-/blob/master/RAG_Project_2_with_pdf_input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install pypdf
!pip install faiss-cpu
!pip install google-generativeai
!pip install tiktoken
!pip install langchain_google-genai

In [ ]:
!pip install streamlit


In [ ]:
!pip install langchain_community

In [4]:

from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
import os

In [ ]:
!pip install chromadb

In [6]:
from langchain.vectorstores import Chroma

In [ ]:
from langchain.document_loaders import PyPDFLoader

# Replace 'my_document.pdf' with the actual name of your PDF file
pdf_path = "my_document.pdf"

def load_pdf(file_path):
    """Loads a PDF file."""
    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split()
    return pages

# Call the function to load the PDF
loaded_docs = load_pdf(pdf_path)

# Print some info to verify
print(f"Loaded {len(loaded_docs)} pages from your PDF.")
print("First 100 words of the first page:")
print(loaded_docs[0].page_content[:100])

In [8]:
len(loaded_docs)

199

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(loaded_docs)

In [10]:
print(len(docs))

473


In [11]:
docs[77]

Document(metadata={'source': 'my_document.pdf', 'page': 34}, page_content='“It’s\tnot\tfor\tyou”\nWe’re\tnot\tsupposed\tto\tsay\tthat.\tWe’re\tcertainly\tnot\tsupposed\tto\twant\tto\tsay\tthat.\nBut\twe\tmust.\n“It’s\tnot\tfor\tyou”\tshows\tthe\tability\tto\trespect\tsomeone\tenough\tthat\tyou’re\tnot\ngoing\tto\twaste\ttheir\ttime,\tpander\tto\tthem,\tor\tinsist\tthat\tthey\tchange\ttheir\tbeliefs.')

In [12]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [13]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings # Import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model= "models/embedding-001")
vector = embeddings.embed_query("hello, world")
vector[:5]

[0.053597815334796906,
 -0.030782219022512436,
 -0.03252851590514183,
 -0.02810630202293396,
 0.022345904260873795]

In [14]:
vectorstore = Chroma.from_documents(docs, embeddings)

In [ ]:
from re import search
retriver = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrived_docs = retriver.invoke("give me two line of summary of my shared pdf")
print(retrived_docs[2].page_content)

In [16]:
llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-exp", temperature=0.3, max_token = 500)

In [17]:
from langchain.chains import RetrievalQA

def ask_question(retriever, question): # add the retriever variable
  """Asks a question and gets an answer using the vector store and Gemini."""
  llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-exp", temperature=0.3, max_token = 500)
  qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever) # pass retriever to the chain
  answer = qa_chain.invoke(question)
  return answer


# Ask your question here!
my_question = "What is the topic of this document?"

# Get the Answer from the document
answer = ask_question(retriever=retriver, question=my_question) # Call the function with the retriever

print(f"Question: {my_question}")
print(f"Answer: {answer}")

Question: What is the topic of this document?
Answer: {'query': 'What is the topic of this document?', 'result': 'Based on the provided table of contents and index, this document appears to be about marketing.\n'}


In [18]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_core.prompts import PromptTemplate

In [20]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_core.prompts import PromptTemplate
# Import ChatPromptTemplate
from langchain.prompts import ChatPromptTemplate # Importing the ChatPromptTemplate class

system_prompt = (
    "you are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, just say that you don't know. "
    "\n\n"
    "{context}"
)
prompt = ChatPromptTemplate(
    [
    ("system", system_prompt),
    ("human", "{input}")
    ]
    )


In [31]:
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain # Importing the necessary function

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriver, question_answer_chain)


In [34]:
response = rag_chain.invoke({"input" : "what is The	Canvas	of	Dreams	and	Desires"})
print(response["answer"])

The Canvas of Dreams and Desires is the title of Chapter Seven. It discusses how marketers, like artists, don't need many colors to paint an original masterpiece. It also discusses the core basket of dreams and desires that people have, such as security, sex, strength, sympathy, and tension. The chapter emphasizes that marketers should begin with dreams and fears, emotional states, and the change they seek to make, rather than with their machines, inventory, or tactics.

